<a href="https://colab.research.google.com/github/rohanrudra55/torrent-to-drive/blob/main/torrent-2-drive-stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader 

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

### Start Connecting

In [ ]:
"""
6881 : TCP Port for torrent client
downloads : To store the downloads
"""
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

### Mount Google Drive
To stream files we need to connect Google Drive. 
(You may need to grant permission)

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
dest_name = "Torrent"
dest = "/content/drive/My Drive/"+dest_name

### Add From Torrent File
You can run this cell to add more files as many times as you want

In [ ]:
from google.colab import files
source = files.upload()
params = {
    "save_path": dest,
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Add From Magnet Link
You can run this cell to add more files as many times as you want. 

Type `exit` when finish adding files.

In [ ]:
params = {"save_path": dest}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


### Start Download

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    # create progress bar for every download
    for _ in downloads
]
# display all the bars
display(*download_bars)


while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        # Select the download and update the bar with torrent status         
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() 
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
